In [1]:
import os
import json
import re
from dotenv import load_dotenv
from typing import Dict, Any, List, Optional, Tuple, Annotated
from IPython.display import Image, display

from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END

In [2]:
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain_core.runnables import RunnableSequence
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.sequential import SequentialChain
import re
import os
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv

In [111]:
# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [45]:
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>{{company_name}} - Official Website</title>
</head>
<body>
    <header>
        <h1>Welcome to {{company_name}}</h1>
        <p>{{company_tagline}}</p>
    </header>
    
    <section class="mission">
        <h2>Our Mission</h2>
        <p>{{company_mission}}</p>
    </section>
    
    <section class="services">
        <h2>Our Services</h2>
        <p>{{company_services}}</p>
    </section>
    
    <footer>
        <p>Contact us at: {{company_email}}</p>
    </footer>
</body>
</html>
"""

In [46]:
analysis_prompt = PromptTemplate(
            input_variables=["html_template"],
            template="""
            You are provided with the content of an HTML file. 
            Your task is to analyze the HTML and identify all data points that are likely candidates for dynamic updates. 
            For example, if the HTML contains a company name, you should include a key called company_name. 
            Your output must be a valid JSON where each key represents a piece of data that might need updating and each value is set to None.

            Instructions:
                1. Analyze the provided HTML content thoroughly.
                2. Identify any elements or data that could be updated, such as company names, addresses, phone numbers, dates, or any other placeholders.
                3. For each identified data point, create a corresponding key in a dictionary. The key should be descriptive (e.g., company_name, address, etc.).
                4. Set the value for each key to None.
                5. Return ONLY the JSON dictionary without any additional text.
            
            HTML Template:
            {html_template}
            """
        )

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)

llm_result = analysis_chain.run(html_template=html_template)

In [47]:
print(llm_result)

```json
{
  "company_name": null,
  "company_tagline": null,
  "company_mission": null,
  "company_services": null,
  "company_email": null
}
```


In [48]:
def clean_json_output(response):
    # Remove code block markers and extract JSON
    cleaned = re.sub(r"```json|```", "", response).strip()
    return json.loads(cleaned)

In [49]:
fields_dict = clean_json_output(llm_result)

In [ ]:
fields_dict 

{'company_name': None,
 'company_tagline': None,
 'company_mission': None,
 'company_services': None,
 'company_email': None}

In [64]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

memory = ConversationBufferMemory(return_messages=True)

# Prompt for gathering user information
interaction_prompt = PromptTemplate(
    input_types={"fields_dict": dict},
    input_variables=["fields_dict", "history"],
    template="""
    Yor are an interactive Chatbot.
    You are provided with a fields_dict that was originally extracted from an HTML file. 
    This dictionary contains keys representing pieces of data (such as company_name, address, etc.) that may need to be updated.
    You will be provided with the user respond. 
    Your task is to interact with the user in order to collect updated values for these fields.

    Instructions:
    1. Begin by informing the user that the following fields were extracted from an HTML file and may require updating.
    2. Ask the user for new values for each field in the fields_dict.
    3. Allow the user to provide updated values for one or more fields, or to indicate that no further information is available.
    4. If the user indicates that they want to proceed with creating the new HTML or if they confirm that they have no additional data to provide, set the key is_complete to True. Otherwise, set it to False.
    5. Return ONLY the JSON dictionary with exactly three keys without any additional text or code:
        - respond: A message to the user summarizing the next steps or confirming their input.
        - fields_dict: The updated dictionary with the new values provided by the user. For any field where the user did not supply an update, retain the original value (which might be None).
        - is_complete: A boolean value (True or False) as described above.
    
    Output Example:
    ```json
    {{
        "respond": "Thank you for your input. Please review the updated fields.",
        "fields_dict": {{
            "company_name": "Your Provided Company Name",
            "address": "Your Provided Address",
            "phone_number": null
        }},
        "is_complete": true
    }}
    ```

    
    fields_dict:
    {{
    {fields_dict}
    }}
    
    
    This is the Conversation history:
    {history}
    """
)
interaction_chain = LLMChain(
    llm=llm, 
    prompt=interaction_prompt,
    memory=memory
)


In [65]:
fields_dict

{'company_name': None,
 'company_tagline': None,
 'company_mission': None,
 'company_services': None,
 'company_email': None}

In [66]:
fields_dict_str = "\n".join([f"{key}: {value}" for key, value in fields_dict.items()])

In [67]:
print(fields_dict_str)

company_name: None
company_tagline: None
company_mission: None
company_services: None
company_email: None


In [68]:
updated_dict = fields_dict.copy()

fields_dict_str = "\n".join([f"{key}: {value}" for key, value in fields_dict.items()])

question = interaction_chain.run(fields_dict=fields_dict_str)

print(question)

```json
{
    "respond": "Hello! We have extracted the following fields from an HTML file. Please provide updated values for each field, or indicate if the current value is correct or if you don't have the information. \n\n- Company Name: None\n- Company Tagline: None\n- Company Mission: None\n- Company Services: None\n- Company Email: None\n\nWhat would you like to update?",
    "fields_dict": {
        "company_name": null,
        "company_tagline": null,
        "company_mission": null,
        "company_services": null,
        "company_email": null
    },
    "is_complete": false
}
```


In [69]:
def clean_json_output(response):
    # Remove code block markers and extract JSON
    cleaned = re.sub(r"```json|```", "", response).strip()
    
    # Replace Python's None with JSON's null
    cleaned = re.sub(r"\bNone\b", "null", cleaned)
    return json.loads(cleaned)

In [70]:
answer_dict = clean_json_output(question)

In [71]:
answer_dict

{'respond': "Hello! We have extracted the following fields from an HTML file. Please provide updated values for each field, or indicate if the current value is correct or if you don't have the information. \n\n- Company Name: null\n- Company Tagline: null\n- Company Mission: null\n- Company Services: null\n- Company Email: null\n\nWhat would you like to update?",
 'fields_dict': {'company_name': None,
  'company_tagline': None,
  'company_mission': None,
  'company_services': None,
  'company_email': None},
 'is_complete': False}

In [72]:
answer_dict['fields_dict']

{'company_name': None,
 'company_tagline': None,
 'company_mission': None,
 'company_services': None,
 'company_email': None}

In [73]:
updated_dict = fields_dict.copy()

In [74]:
updated_dict

{'company_name': None,
 'company_tagline': None,
 'company_mission': None,
 'company_services': None,
 'company_email': None}

In [75]:
updated_dict = fields_dict.copy()
memory = ConversationBufferMemory(return_messages=True)
interaction_chain = LLMChain(
    llm=llm, 
    prompt=interaction_prompt,
    memory=memory
)

fields_dict_str = "\n".join([f"{key}: {value}" for key, value in updated_dict.items()])
print('*'*50, "Histoy", '*'*50)
print(interaction_chain.memory.chat_memory)

response = interaction_chain.run(fields_dict=fields_dict_str)

answer_dict = clean_json_output(response)
llm_respond = answer_dict['respond']
updated_dict = answer_dict['fields_dict']
is_complete = answer_dict['is_complete']

print('*'*50, "AI", '*'*50)
print(f"AI: {llm_respond}")
print('*'*50, "updated_dict", '*'*50)
print(updated_dict)


# Get user response
user_input = 'Company name is electro pi'.strip()
print('*'*50, "Human", '*'*50)
print(f"Human: {user_input}")
interaction_chain.memory.chat_memory.add_user_message(user_input)

print('*'*50, "Histoy", '*'*50)
print(interaction_chain.memory.chat_memory)

************************************************** Histoy **************************************************

************************************************** AI **************************************************
AI: I have extracted the following fields from an HTML file: company_name, company_tagline, company_mission, company_services, and company_email. Could you please provide updated values for these fields? If you don't have information for a specific field, you can skip it.
************************************************** updated_dict **************************************************
{'company_name': None, 'company_tagline': None, 'company_mission': None, 'company_services': None, 'company_email': None}
************************************************** Human **************************************************
Human: Company name is electro pi
************************************************** Histoy **************************************************
Human: company_name: Non

In [79]:
response = interaction_chain.run(fields_dict=fields_dict_str)

In [81]:
print(response)

```json
{
    "respond": "Thank you. I have updated the company_name to 'electro pi'. Please provide values for the remaining fields: company_tagline, company_mission, company_services, and company_email. Let me know if you want to proceed with the current information.",
    "fields_dict": {
        "company_name": "electro pi",
        "company_tagline": null,
        "company_mission": null,
        "company_services": null,
        "company_email": null
    },
    "is_complete": false
}
```


In [78]:
interaction_chain.memory.chat_memory.messages

[HumanMessage(content='company_name: None\ncompany_tagline: None\ncompany_mission: None\ncompany_services: None\ncompany_email: None', additional_kwargs={}, response_metadata={}),
 AIMessage(content='```json\n{\n    "respond": "I have extracted the following fields from an HTML file: company_name, company_tagline, company_mission, company_services, and company_email. Could you please provide updated values for these fields? If you don\'t have information for a specific field, you can skip it.",\n    "fields_dict": {\n        "company_name": null,\n        "company_tagline": null,\n        "company_mission": null,\n        "company_services": null,\n        "company_email": null\n    },\n    "is_complete": false\n}\n```', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Company name is electro pi', additional_kwargs={}, response_metadata={})]

In [ ]:
updated_dict = fields_dict.copy()
memory = ConversationBufferMemory(return_messages=True)

while True:
    fields_dict_str = "\n".join([f"{key}: {value}" for key, value in updated_dict.items()])

    response = interaction_chain.run(fields_dict=fields_dict_str)
    
    answer_dict = clean_json_output(response)
    llm_respond = answer_dict['respond']
    updated_dict = answer_dict['fields_dict']
    is_complete = answer_dict['is_complete']
    
    print('*'*50, "AI", '*'*50)
    print(f"AI: {llm_respond}")
    print('*'*50, "updated_dict", '*'*50)
    print(updated_dict)
    
    if is_complete:
        break
    
    # Get user response
    user_input = input().strip()
    print('*'*50, "Human", '*'*50)
    print(f"Human: {user_input}")
    interaction_chain.memory.chat_memory.add_user_message(user_input)

************************************************** AI **************************************************
AI: Okay, I still need the company_tagline, company_mission, company_services, and company_email. Please provide the values or let me know if you are ready to proceed.
************************************************** updated_dict **************************************************
{'company_name': 'electro pi', 'company_tagline': None, 'company_mission': None, 'company_services': None, 'company_email': None}
************************************************** Human **************************************************
Human: campany tag line is work with us
************************************************** AI **************************************************
AI: Thank you! I've updated the company_tagline to 'work with us'. Please provide values for company_mission, company_services, and company_email. Or, let me know if you are ready to proceed with the current information.
********

In [213]:
interaction_chain.memory.chat_memory.add_user_message("")

In [214]:
interaction_chain.memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='company_name: None\ncompany_tagline: None\ncompany_mission: None\ncompany_services: None\ncompany_email: None', additional_kwargs={}, response_metadata={}), AIMessage(content='```json\n{\n    "respond": "We have extracted the following fields from an HTML file: company_name, company_tagline, company_mission, company_services, and company_email. Please provide updated values for these fields, or indicate if no update is needed. Let\'s start with the company name. What is the company name?",\n    "fields_dict": {\n        "company_name": null,\n        "company_tagline": null,\n        "company_mission": null,\n        "company_services": null,\n        "company_email": null\n    },\n    "is_complete": false\n}\n```', additional_kwargs={}, response_metadata={}), HumanMessage(content='company_name: None\ncompany_tagline: None\ncompany_mission: None\ncompany_services: None\ncompany_email: None', additional_kwargs={}, response_metad

In [36]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

memory = ConversationBufferMemory(return_messages=True)

# Prompt for gathering user information
interaction_prompt = PromptTemplate(
    input_variables=['text','history'],
    template="""
    Yor are an interactive Chatbot.
    {text}
    
    this is the Conversation history:
    {history}
    """
)
interaction_chain = LLMChain(
    llm=llm, 
    prompt=interaction_prompt,
    memory=memory
)


In [38]:
interaction_chain.memory.chat_memory.add_user_message('my name is esmail')

In [39]:
interaction_chain.memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='my name is esmail', additional_kwargs={}, response_metadata={})])

In [29]:
result = interaction_chain.run(text='how are you, my name is esmail')

In [30]:
result

"I'm doing well, thanks for asking! How can I help you today, Esmail?"

In [40]:
result = interaction_chain.run(text='what is my name?')

In [41]:
result

'Based on the conversation history, your name is Esmail.'

In [3]:
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>{{company_name}} - Official Website</title>
</head>
<body>
    <header>
        <h1>Welcome to {{company_name}}</h1>
        <p>{{company_tagline}}</p>
    </header>
    
    <section class="mission">
        <h2>Our Mission</h2>
        <p>{{company_mission}}</p>
    </section>
    
    <section class="services">
        <h2>Our Services</h2>
        <p>{{company_services}}</p>
    </section>
    
    <footer>
        <p>Contact us at: {{company_email}}</p>
    </footer>
</body>
</html>
"""

In [4]:
fields_dict = {
  'company_name': 'electro pi',
  'company_tagline': 'come to us',
  'company_mission': 'help everyone',
  'company_services': 'ai solutions',
  'company_email': 'bb@gmail.com'}

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

update_prompt = PromptTemplate(
            input_variables=["html_template", "fields_dict"],
            template="""
            You are a system that updates HTML templates by replacing placeholders with actual values.
            
            Given the following HTML template:
            {html_template}
            
            And the following dictionary of placeholders and their values:
            {fields_dict}
            
            Return the updated HTML template with all placeholders replaced with their corresponding values.
            If a placeholder doesn't have a corresponding value (None), leave it unchanged.
            """
        )


analysis_chain = LLMChain(llm=llm, prompt=update_prompt)

llm_result = analysis_chain.run(html_template=html_template, fields_dict=fields_dict)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10484\2868247582.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  analysis_chain = LLMChain(llm=llm, prompt=update_prompt)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10484\2868247582.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_result = analysis_chain.run(html_template=html_template, fields_dict=fields_dict)


In [7]:
print(llm_result)

```html
<!DOCTYPE html>
<html>
<head>
    <title>electro pi - Official Website</title>
</head>
<body>
    <header>
        <h1>Welcome to electro pi</h1>
        <p>come to us</p>
    </header>

    <section class="mission">
        <h2>Our Mission</h2>
        <p>help everyone</p>
    </section>

    <section class="services">
        <h2>Our Services</h2>
        <p>ai solutions</p>
    </section>

    <footer>
        <p>Contact us at: bb@gmail.com</p>
    </footer>
</body>
</html>
```
